# City simulation

In [7]:
import matplotlib.animation
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

from random import choice
import itertools
from enum import Enum

import agentpy as ap

## Create Types

In [8]:
class Moves(Enum):
    RIGHT = (0, 1)
    LEFT = (0, -1)
    DOWN = (1, 0)
    UP = (-1, 0)

class Colors(Enum):
    YELLOW = 1
    GREEN = 2
    RED = 3
    BLUE = 4

## Create Car and Traffic light agents

In [9]:
class CarAgent(ap.Agent):
    def setup(self):
        self.current_move_type = choice(list(Moves))
        self.has_light = False
    
    def move(self):
        if self.__check_can_pass(self.current_move_type.value):
            self.model.street.move_by(self, self.current_move_type.value)

        if not self.has_light:
            self.__check_light()
        else:
            self.__check_passed_light()
    
    def __check_light(self):
        width = self.model.width
        height = self.model.height
        pos = self.model.street.positions[self]
        
        if self.current_move_type in (Moves.DOWN, Moves.UP):
            for i in range(pos[0], height):
                for agent in self.model.street.agents[i, pos[1]]:
                    if agent.type == "TrafficLightAgent":
                        self.has_light = True
                        agent.add_car()
                        return
        else:
            for i in range(pos[1], width):
                for agent in self.model.street.agents[pos[0], i]:
                    if agent.type == "TrafficLightAgent":
                        self.has_light = True
                        agent.add_car()
                        return
    
    def __check_passed_light(self):
        pos = self.model.street.positions[self]

        for agent in self.model.street.agents[pos]:
            if agent.type == "TrafficLightAgent":
                self.has_light = False
                agent.remove_car()
                break
    
    def __get_new_pos(self, move):
        pos = self.model.street.positions[self]

        return tuple(map(sum, zip(pos, move)))

    def __check_can_pass(self, move):
        width = self.model.width
        height = self.model.height
        new_pos = self.__get_new_pos(move)

        for agent in self.model.street.agents[new_pos[0] % height, new_pos[1] % width]:
            if agent.type == 'TrafficLightAgent' and agent.is_red():
                return False
            if agent.type == 'CarAgent':
                return False
        return True



class TrafficLightAgent(ap.Agent):
    def setup(self):
        self.light_color = Colors.YELLOW
    
        self.green_time = 0
        self.red_time = 0
        self.cars = 0

        self.time = 15
        self.countdown = 0
        self.reduce_time_times_car = 0.93

    def add_car(self):
        if self.is_red():
            self.red_time *= self.reduce_time_times_car

        self.cars += 1
    
    def remove_car(self):
        if self.cars > 0:
            self.cars -= 1

        if self.cars == 0:
            self.change_to_yellow()

    def reduce_time(self):
        if self.cars == 0:
            self.change_to_yellow()

        pos = self.model.street.positions[self]
        #print(f"{self.cars}, {pos}")

        if self.is_yellow():
            for agent in self.model.street.agents[pos[0] - 1, pos[1] + 1]:
                if agent.type == "TrafficLightAgent" and agent.is_green():
                    self.change_to_red(self.time)
                    return
                elif agent.type == "TrafficLightAgent" and self.cars > 0:
                    self.change_to_green(self.time)
                    return
            for agent in self.model.street.agents[pos[0] + 1, pos[1] - 1]:
                if agent.type == "TrafficLightAgent" and agent.is_green():
                    self.change_to_red(self.time)
                    return
                elif agent.type == "TrafficLightAgent" and self.cars > 0:
                    self.change_to_green(self.time)
                    return
        elif self.is_red():
            if self.red_time > 0:
                self.red_time -= 1
                return

            for agent in self.model.street.agents[pos[0] - 1, pos[1] + 1]:
                if agent.type == "TrafficLightAgent" and self.red_time <= 0:
                    agent.change_to_red()
                    self.change_to_green()
                    return
            for agent in self.model.street.agents[pos[0] + 1, pos[1] - 1]:
                if agent.type == "TrafficLightAgent" and self.red_time <= 0:
                    agent.change_to_red()
                    self.change_to_green()
                    return
        else:
            if self.green_time > 0:
                self.green_time -= 1
                return
            
            for agent in self.model.street.agents[pos[0] - 1, pos[1] + 1]:
                if agent.type == "TrafficLightAgent" and self.green_time <= 0:
                    agent.change_to_green()
                    self.change_to_red(self.time)
                    return
            for agent in self.model.street.agents[pos[0] + 1, pos[1] - 1]:
                if agent.type == "TrafficLightAgent" and self.green_time <= 0:
                    agent.change_to_green()
                    self.change_to_red(self.time)
                    return
    
    def change_to_red(self, time = 0):
        if time == 0: time = self.time
        if self.light_color != Colors.RED:
            self.red_time = time

            self.light_color = Colors.RED
            self.color = self.light_color.value

    def change_to_green(self, time = 0):
        if time == 0: time = self.time
        if self.light_color != Colors.GREEN:
            self.green_time = time

            self.light_color = Colors.GREEN
            self.color = self.light_color.value
    
    def change_to_yellow(self):
        self.green_time = 0
        self.red_time = 0
        
        self.light_color = Colors.YELLOW
        self.color = self.light_color.value
    
    def is_green(self):
        return self.light_color == Colors.GREEN

    def is_yellow(self):
        return self.light_color == Colors.YELLOW

    def is_red(self):
        return self.light_color == Colors.RED

## Create City Model

In [10]:
class CityModel(ap.Model):
    def setup(self):
        x_separations = self.p.lights_separation["X"]
        y_separations = self.p.lights_separation["Y"]

        self.width = int(self.p["size"][0])
        self.height = int(self.p["size"][1])

        x_positions = range(x_separations, self.height, x_separations)
        y_positions = range(y_separations, self.width, y_separations)

        # create lights
        lights_locations = list(itertools.product(x_positions, y_positions))
        lights_locations = [(location[0], location[1] - 1) for location in lights_locations] + [(location[0] - 1, location[1]) for location in lights_locations]

        n_lights = len(lights_locations)
        self.lights = ap.AgentList(self, n_lights, TrafficLightAgent)

        # create cars
        n_cars = int(self.p["cars"])
        self.cars = ap.AgentList(self, n_cars, CarAgent)

        # initialize colors of agents
        self.lights.color = Colors.YELLOW.value
        self.cars.color = Colors.BLUE.value

        self.lights.time = 15

        # create street
        self.street = ap.Grid(self, [self.width, self.height], track_empty=True, torus=True)

        # add lights
        self.street.add_agents(self.lights, lights_locations)

        # add cars
        valid_spawn_points = [(0, x) for x in x_positions] + [(y, 0) for y in y_positions]

        spawn_points = [choice(valid_spawn_points) for _ in self.cars]

        self.__spawn_cars(spawn_points)

    def __spawn_cars(self, spawn_points):
        for car, spawn_point in zip(self.cars, spawn_points):
            if spawn_point[0] == 0:
                car.current_move_type = Moves.DOWN
            elif spawn_point[1] == 0:
                car.current_move_type = Moves.RIGHT
        
        self.street.add_agents(self.cars, spawn_points)

    def step(self):
        for car in self.cars:
            car.move()
        
        for light in self.lights:
            light.reduce_time()

    def end(self):
        pass

## Create parameters

In [11]:
# separation can't be less than three

parameters = {
    "size": [30, 30],
    "cars": 30,
    "lights_separation": {"X": 10, "Y": 10},
    "steps": 300,
}

## Start animation

In [12]:
def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('color')
    color_dict = {Colors.BLUE.value:'#00FFFF', Colors.RED.value:'#d62c2c', Colors.YELLOW.value:'#FFFF00', Colors.GREEN.value:'#008000', None:'#dfdfdf'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a City\n"
                 f"Time-step: {model.t}")

fig, ax = plt.subplots() 
model = CityModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))